**NOTE: This notebook is written for the Google Colab platform. However it can also be run (possibly with minor modifications) as a standard Jupyter notebook.**

In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install skorch

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
from PIL import Image
from torchvision import models
from torchvision import transforms
from skorch import NeuralNetClassifier, NeuralNet
import torch

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
!mkdir -p data
!mkdir -p output
!wget -nc -O data/cat.jpg https://www.dropbox.com/s/a5ux951zo01gd5z/cat.jpg?dl=1
!wget -nc -O data/imagenet_classes https://www.dropbox.com/s/ma25i7w3jpqex2a/imagenet_classes?dl=1

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }
with open("data/imagenet_classes", "r") as file:
    classes = [c[:-1] for c in file.readlines()]
    
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

def preproc_image(img):
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)
    return batch_t

# Using a Pre-trained Classifier

This notebook will show a very simple example of loading and using a classifier pre-trained on ImageNet. We will show how it can be used to classify new images.
# Loading the Model

In the previous examples we have defined a class for our own neural net and specified its architecture. Now we will instead use one of the predefined models with pretrained weights. These models are available in the ``torchvision.models`` package. We will use architecture ``densenet169`` in particular and specify the argument ``pretrained=True`` when creating an instance. This means that the network is to be initialized with weights pretrained on the ImageNet dataset. If we are calling this piece of code for the first time, the weight will need to be downloaded over the internet first.

To load a model we could also use the ``torch.hub`` tool – in that case the model would not have to be built into PyTorch. We could instead load it directly from one of the special GitHub repositories that support this. For instance, we could load the well-known EfficientNetB0 using the following code:
```python
torch.hub.load('rwightman/gen-efficientnet-pytorch', 'efficientnet_b0', pretrained=True)
```

In [ ]:
module = models.densenet169(pretrained=True)

When creating a classifier we will now only need to specify the instance of our model and specify the device that we want to use. In our case we will, however, need to append a softmax layer to the end of the network – networks from ``torchvision.models`` do not have a softmax layer because the loss function they were trained incorporated it. We are not going to train the model and therefore we also need to call function ``initialize``, which will make the network ready for inference (the pretrained weights will of course not be replaced).

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

net = NeuralNetClassifier(
    torch.nn.Sequential(module, torch.nn.Softmax()),
    device=device,
)

net.initialize();

## Using the Model

We load and preprocess the image that we want to classify.

In [ ]:
img = Image.open("data/cat.jpg")
display(img)

Before plugging the image into the network we will need to apply some slight preprocessing using function ``preproc_image``. This is because we need to ensure that images are preprocessed in the same way as when the network was trained.

In [ ]:
img_prep = preproc_image(img)
y = net.predict(img_prep)

The network will return the number of the class. The list of class labels is stored in the ``classes`` list (it was read from a file in the auxiliary code section), so we only need to index it using the output of our network:

In [ ]:
classes[y.item()]

## Displaying the Top-5 Predictions

Given that our network actually predicts class probabilities, we might want to known about those and not just about the label of the most probable class. In fact, let's display the top-5 predictions and their probabilities. This will give us a better idea of how confident the neural network is about its prediction and whether the other, less probable predictions make any sense or not.

Our neural classifier has an interface for querying about probabilities: we can use the ``predict_proba`` method, which is also available in a number of different ``scikit-learn`` classifiers.

In [ ]:
proba = net.predict_proba(img_prep)

Once we have got the probabilities, all we need to do is to sort them and identify the classes with the top-5 probabilities. To this end we will define a small auxiliary function and apply it to the probabilities.

In [ ]:
def decode_proba(proba, top=5):
    proba = proba.ravel()
    ind = np.argsort(proba)
    
    for c in reversed(ind[-top:]):
        print("{}:\t{} ({})".format(
            np.array2string(proba[c], precision=5),
            classes[c], c))

In [ ]:
decode_proba(proba)

---

## Task 1: Predictions about Other Images

**Try to apply the same procedure to a different image.**

Note: New images can be uploaded **directly through the notebook interface** or else using:
```python
from google.colab import files
content_img = files.upload()
filename = list(content_img)[0]
```

---

The list of classes that the network is supposed to be able to classify can be found in file ``data/imagenet_classes``

In [ ]:
for ic, c in enumerate(classes):
    print("{}:\t{}".format(ic, c))